In [1]:
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from datetime import datetime

import sys
sys.path.append('../')

from nc_explorer import NcExplorer
exp = NcExplorer()

sen3r-NcExplorer: from mpl_toolkits.basemap import Basemap FAILED!You can still proceed without plotting any maps.
Declaring class instance from: SEN3R:Nc_Explorer
Input NetCDF file folder not set. Proceed at your own risk.


In [18]:
def dfcompat(df1, df2, colname):
    """
    Compare two pd.DataFrames given a common column in both of them,
    then, return two new DFs containing only the rows where they mach in
    the given column and the set used to match them.
    """
    set_df1 = set(df1[colname])
    set_df2 = set(df2[colname])
    matches = set_df1.intersection(set_df2)
    
    df1_match = df1[df1[colname].isin(matches)]
    df2_match = df2[df2[colname].isin(matches)]
    
    return df1_match, df2_match, matches

In [4]:
df = pd.read_excel('/d_drive_data/processing/OTHER_PLATFORMS/andre_field_data_s3compat.xlsx')

In [5]:
df.head()

,Date,Oa01,Oa02,Oa03,Oa04,Oa05,Oa06,Oa07,Oa08,Oa09,...,Oa12,Oa13,Oa14,Oa15,Oa16,Oa17,Oa18,Oa19,Oa20,Oa21
0,2017-12-05 00:35:23,0.011827,0.012984,0.016698,0.021469,0.024181,0.032995,0.039927,0.041732,0.042042,...,0.025976,0.027293,0.027368,0.026883,0.027374,0.018107,0.015585,0.013942,0.007581,NaN
1,2017-12-05 00:35:37,0.012268,0.013483,0.017370,0.022365,0.025190,0.034345,0.041453,0.043174,0.043481,...,0.026294,0.027590,0.027648,0.027153,0.027701,0.018230,0.015661,0.013852,0.007402,NaN
2,2017-12-05 00:35:37,0.012269,0.013536,0.017615,0.022850,0.025796,0.035310,0.042599,0.044378,0.044681,...,0.026353,0.027693,0.027750,0.027250,0.027776,0.017920,0.015336,0.013675,0.007709,NaN
3,2017-12-05 00:35:37,0.012653,0.013864,0.017740,0.022686,0.025493,0.034596,0.041625,0.043392,0.043681,...,0.026732,0.028057,0.028110,0.027617,0.028149,0.018605,0.016029,0.014230,0.007931,NaN
4,2017-12-05 00:35:37,0.012484,0.013745,0.017753,0.022880,0.025775,0.035144,0.042338,0.044145,0.044465,...,0.026700,0.028063,0.028128,0.027630,0.028143,0.018358,0.015732,0.014046,0.007752,NaN


In [6]:
df['pydt'] = pd.to_datetime(df['Date'])

In [44]:
# Days of observed ground truth
field_days = list(df['pydt'].unique())
field_days[0]

numpy.datetime64('2017-12-05T00:35:23.000000000')

In [46]:
# convert from datetime64 to datetime.datetime
field_days = [f.astype('M8[ms]').astype('O') for f in field_days]
# drop hour, minute, second
field_days_notime = [e.replace(hour=0, minute=0, second=0) for e in field_days]
field_days_notime[0]

datetime.datetime(2017, 12, 5, 0, 0)

In [33]:
l2wfr = os.listdir('/d_drive_data/S3/L2_WFR_subset/MANACAPURU')
l2wfr_dates = [s3img.split('____')[1].split('_')[0] for s3img in l2wfr]
l2wfr_dates[0:5]

['20160425T134227',
 '20160428T140436',
 '20160429T133846',
 '20160502T140055',
 '20160503T133505']

In [34]:
# Days of S3 orbital images
l2wfr_pydts = [datetime.strptime(img_date, '%Y%m%dT%H%M%S') for img_date in l2wfr_dates]
l2wfr_pydts[0]

datetime.datetime(2016, 4, 25, 13, 42, 27)

In [35]:
# removes hour:minute:seconds from Sentinel-3 data.
l2wfr_pydts_notime = [e.replace(hour=0, minute=0, second=0) for e in l2wfr_pydts]
l2wfr_pydts_notime[0]

datetime.datetime(2016, 4, 25, 0, 0)

In [47]:
# Turn both data in a SET in order to compare
set_field = set(field_days_notime)
set_s3 = set(l2wfr_pydts_notime)
matches = set_field.intersection(set_s3)

In [48]:
len(set_field)

9

In [49]:
len(set_s3)

892

In [51]:
for m in matches:
    print(m)

2017-12-05 00:00:00
2019-03-13 00:00:00
2019-05-17 00:00:00
2019-02-14 00:00:00
2019-04-21 00:00:00
2018-05-25 00:00:00


In [52]:
matches = ['S3A_OL_2_WFR____20171205T141218_20171205T141518_20171206T200523_0179_025_167_3059_MAR_O_NT_002_subset.txt',
'S3A_OL_2_WFR____20190313T140838_20190313T141138_20190314T214804_0179_042_224_3060_MAR_O_NT_002_subset.txt',
'S3B_OL_2_WFR____20190517T134411_20190517T134711_20190518T190927_0180_025_238_3060_MAR_O_NT_002_subset.txt',
'S3A_OL_2_WFR____20190214T140841_20190214T141141_20190215T231651_0179_041_224_3060_MAR_O_NT_002_subset.txt',
'S3A_OL_2_WFR____20190421T135732_20190421T140032_20190422T221041_0179_044_010_3060_MAR_O_NT_002_subset.txt',
'S3A_OL_2_WFR____20180525T133838_20180525T134138_20180526T203409_0179_031_295_3060_MAR_O_NT_002_subset.txt']

In [53]:
matches

['S3A_OL_2_WFR____20171205T141218_20171205T141518_20171206T200523_0179_025_167_3059_MAR_O_NT_002_subset.txt',
 'S3A_OL_2_WFR____20190313T140838_20190313T141138_20190314T214804_0179_042_224_3060_MAR_O_NT_002_subset.txt',
 'S3B_OL_2_WFR____20190517T134411_20190517T134711_20190518T190927_0180_025_238_3060_MAR_O_NT_002_subset.txt',
 'S3A_OL_2_WFR____20190214T140841_20190214T141141_20190215T231651_0179_041_224_3060_MAR_O_NT_002_subset.txt',
 'S3A_OL_2_WFR____20190421T135732_20190421T140032_20190422T221041_0179_044_010_3060_MAR_O_NT_002_subset.txt',
 'S3A_OL_2_WFR____20180525T133838_20180525T134138_20180526T203409_0179_031_295_3060_MAR_O_NT_002_subset.txt']